# **OPC Data - Last Four Weeks**

*1. Select information from SQL server*

*2. Download CSV data from SQL server*

*3. Upload CSV data here*



*- Press ctrl+f9 to rull all data. Make sure that current team and file name is accurate -*

**List of current team members**

In [ ]:
current_team = ['Lara Crisp', 'Shianne Elliott', 'Conan Cecconie', 'Jonathan Reale', 'Jacob Friedman', 'Taylor Keyes', 'Beau Thomas']

**The name of the uploaded file should EXACTLY match the one below**

In [ ]:
file_name = 'Program Ledger By Personnel.csv'

## **OPC Team Graphs**

**- The following cell will notify if a date or person is not found in the uploaded file -**

In [ ]:
#@title
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import datetime, date, time, timezone, timedelta

BGV = pd.read_csv(file_name)
BGV['TourDate1'] = pd.to_datetime(BGV['TourDate1'])
BGV.drop_duplicates(subset=['PersonnelNameTitle', 'TourID2'], inplace=True)

alt_BGV = BGV

# Compute 4 week date range from today
today = date.today()
today = today.strftime("%Y-%m-%d")
today = datetime.strptime(today, '%Y-%m-%d')
two_weeks_ago = today - timedelta(days=28)   #go back 28 days aka 4 weeks
two_weeks_ago = two_weeks_ago.strftime("%Y-%m-%d")
today = today.strftime("%Y-%m-%d")

BGV = BGV.loc[(BGV['TourDate1'] >= two_weeks_ago) & (BGV['TourDate1'] <= today)]

opc_team = pd.DataFrame()
for individual in range(len(current_team)):
  opc_team = opc_team.append(BGV.loc[BGV['PersonnelNameTitle'] == str(current_team[individual])+': Tour Booker'])

BGV = BGV.sort_values('TourDate1')
daterange = pd.date_range(start = two_weeks_ago, end = today, freq='d')

opc_team_showed = opc_team.loc[opc_team['TourStatus2'] == 'Showed']

# Compute 6 weeks back for 14 day average
two_weeks_ago = datetime.strptime(two_weeks_ago, '%Y-%m-%d')
six_weeks_ago = two_weeks_ago - timedelta(days=13)   #go back 13 days
time = six_weeks_ago.strftime("%Y-%m-%d")
two_weeks_ago = two_weeks_ago.strftime('%Y-%m-%d') # Convert back to str

for i in range(len(daterange)):
  if daterange[i].strftime('%Y-%m-%d') not in BGV['TourDate1'].astype('str').unique():
    print('NOTICE: {} is not in uploaded file'.format(daterange[i].strftime('%Y-%m-%d')))

# Create graph of last 4 weeks
from matplotlib.ticker import MaxNLocator
import matplotlib.dates as mdates

for individual in range(len(current_team)):
  name = current_team[individual]
  if name+': Tour Booker' in opc_team['PersonnelNameTitle'].unique():
    fig, ax = plt.subplots(figsize=(15,4))
    
    individual_showed = opc_team_showed.loc[opc_team_showed['PersonnelNameTitle'] == name+': Tour Booker']

    df1 = individual_showed.loc[(individual_showed['ContractStatus1'] != 'Active\r\nActive') & (individual_showed['ContractStatus1'] != 'Suspense\r\nSuspense')]

    df2 = individual_showed.loc[individual_showed['CommissionVolume'] > 0]
    df2 = df2.loc[(df2['ContractStatus1'] == 'Active\r\nActive') | (df2['ContractStatus1'] == 'Suspense\r\nSuspense')]

    #daterange = datetime.strptime(daterange, '%Y-%m-%d')
    individual_daterange = pd.DataFrame(index=daterange)
    individual_daterange['Tour Count'] = individual_showed['TourDate1'].value_counts()
    individual_daterange['Tour Count'] = individual_daterange['Tour Count'].fillna(0)
    individual_daterange['Tour Date'] = individual_daterange.index.strftime('%Y-%m-%d')
    individual_daterange['Average'] = individual_daterange['Tour Count'].rolling(14).mean()

    individual_daterange['Purchase Count'] = df2['TourDate1'].value_counts()
    individual_daterange['Shown Count'] = df1['TourDate1'].value_counts()
    individual_daterange['Purchase Count'] = individual_daterange['Purchase Count'].fillna(0)
    individual_daterange['Shown Count'] = individual_daterange['Shown Count'].fillna(0)

    alt_daterange = pd.date_range(start = six_weeks_ago, end = today, freq='d')
    alt_individual_daterange = pd.DataFrame(index=alt_daterange)

    alt_individual_showed = alt_BGV.loc[(alt_BGV['PersonnelNameTitle'] == name+': Tour Booker') & (alt_BGV['TourStatus2'] == 'Showed')]
    alt_individual_daterange['Tour Count'] = alt_individual_showed['TourDate1'].value_counts()
    alt_individual_daterange['Tour Count'] = alt_individual_daterange['Tour Count'].fillna(0)
    alt_individual_daterange['Day'] = alt_individual_daterange.index.strftime('%d')

    x = alt_individual_daterange['Tour Count'].rolling(14).mean().dropna()
    individual_daterange['Rolling Average'] = x.to_list()

    individual_daterange.plot('Tour Date', 'Tour Count', kind='bar', color='lightskyblue', alpha=0.75, ax=ax, zorder=3)
    individual_daterange.plot('Tour Date', 'Purchase Count', kind='bar', color='forestgreen', ax=ax, alpha=0.5, zorder=5)
    individual_daterange.plot('Tour Date', 'Rolling Average', linewidth=3, color='#1f77b4', ax=ax, zorder=6).grid(axis='y', zorder=0, alpha=0.5)

    count = individual_daterange['Tour Count'].sum().astype(int)
    average = individual_daterange['Tour Count'].fillna(0).mean()
    plt.axhline(y=average, color='lightcoral', zorder=4)
    ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    [l.set_visible(False) for (i,l) in enumerate(ax.xaxis.get_ticklabels()) if i % 2 != 0]

    plt.xlabel('')
    plt.xticks(rotation=90)
    plt.title('   '+str(name)+' (Total: '+str(count)+')', weight='bold', loc='left', fontsize=12)
    plt.title('Tour Count Average from '+str(two_weeks_ago)+' to '+str(today)+'   ', loc='right', fontsize=12)
    plt.legend(['14 Day Average', 'Overall Average: '+str(round(average,2))])

  else:
    print("Can't find data for " + name)